In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
import anndata
import umap.umap_ as umap
import leidenalg
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn.cluster import KMeans
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics import pairwise_distances
from scipy.stats import zscore
from matplotlib.colorbar import ColorbarBase
import matplotlib.colors as mcolors

%matplotlib inline

In [2]:
plt.style.use('dark_background')

### Xenium Cell Area Analysis

In [3]:
xenium_scdata = sc.read_h5ad(r"D:\SMI-0214_DonCleveland_UCSD\SMI-0214_DonCleveland_UCSD\Xenium_mouse_filtered.h5ad")
xenium_scdata

AnnData object with n_obs × n_vars = 547139 × 248
    obs: 'cell_id', 'Sample ID', 'Number of genes', 'Number of transcripts', 'leiden', 'cluster_names', 'celltype', 'cell_area'
    uns: 'Sample ID_colors', 'cell_id_colors', 'celltype_colors', 'cluster_names_colors', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups'
    obsm: 'X_pca', 'X_raw', 'X_spatial', 'X_spatial_rescaled', 'X_spatial_rescaled_flipped', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [4]:
xenium_scdata.obs

,cell_id,Sample ID,Number of genes,Number of transcripts,leiden,cluster_names,celltype,cell_area
1,2,C3m1,12.0,12,12,Micro-PVM,Microglia,50.439531
2,3,C3m1,34.0,34,18,L5 PT CTX,Cortical Neurons V,251.430000
4,5,C3m1,6.0,6,15,Oligio,Oligo II,59.199844
5,6,C3m1,16.0,16,2,Astro,Astro II,51.252344
6,7,C3m1,30.0,30,2,Astro,Astro II,160.440156
...,...,...,...,...,...,...,...,...
556312,62685,R61,159.0,159,1,Astro,Astro I,304.353125
556313,62686,R61,97.0,97,0,Endo,Endo,378.093281
556314,62687,R61,86.0,86,5,,Striatal Neurons I,188.391875
556315,62688,R61,5.0,5,2,Astro,Astro II,824.056406


In [17]:
xenium_df = xenium_scdata.obs[['Sample ID', 'cell_area']]
xenium_df = xenium_df.rename(columns = {'cell_area':'Sample Area'})
xenium_df

,Sample ID,Sample Area
1,C3m1,50.439531
2,C3m1,251.430000
4,C3m1,59.199844
5,C3m1,51.252344
6,C3m1,160.440156
...,...,...
556312,R61,304.353125
556313,R61,378.093281
556314,R61,188.391875
556315,R61,824.056406


In [18]:
cell_area_by_sample_xenium = xenium_df.groupby('Sample ID').sum()
cell_area_by_sample_xenium

,Sample Area
Sample ID,
C3m1,1.715546e+07
C3m2,1.697087e+07
C3m3,1.269861e+07
C24m1,1.717815e+07
C24m2,1.796903e+07
C24m3,1.749998e+07
HD24m1,1.641413e+07
HD24m2,1.901652e+07
HD24m3,1.437131e+07


### Cosmx Cell Area Analysis

In [7]:
cosmx_scdata = sc.read_h5ad(r"D:\SMI-0214_DonCleveland_UCSD\SMI-0214_DonCleveland_UCSD\Cosmx_mouse_filtered.h5ad")
cosmx_scdata

AnnData object with n_obs × n_vars = 132721 × 950
    obs: 'total_counts', 'leiden', 'celltype', 'Area', 'Sample ID'
    var: 'mean', 'std'
    uns: 'Sample ID_colors', 'celltype_colors', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'pca'
    obsm: 'X_pca', 'X_raw', 'X_spatial', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [8]:
cosmx_scdata.obs

,total_counts,leiden,celltype,Area,Sample ID
1,14.0,8,Endo,575.28,175_3 HD Model
2,26.0,0,Astro,758.64,175_3 HD Model
3,12.0,3,Cortical Neurons I,1185.12,175_3 HD Model
4,171.0,7,Oligo III,1413.00,175_3 HD Model
5,276.0,14,Ependymal Cells,1473.36,175_3 HD Model
...,...,...,...,...,...
134377,60.0,7,Oligo III,170.64,7_2 Cntr
134378,42.0,8,Endo,571.44,7_2 Cntr
134379,308.0,0,Astro,1056.36,7_2 Cntr
134380,108.0,7,Oligo III,399.00,7_2 Cntr


In [19]:
cosmx_df = cosmx_scdata.obs[['Area', 'Sample ID']]
cosmx_df = cosmx_df.rename(columns = {'Area':'Sample Area'})
cosmx_df

,Sample Area,Sample ID
1,575.28,175_3 HD Model
2,758.64,175_3 HD Model
3,1185.12,175_3 HD Model
4,1413.00,175_3 HD Model
5,1473.36,175_3 HD Model
...,...,...
134377,170.64,7_2 Cntr
134378,571.44,7_2 Cntr
134379,1056.36,7_2 Cntr
134380,399.00,7_2 Cntr


In [20]:
cell_area_by_sample_cosmx = cosmx_df.groupby('Sample ID').sum()
cell_area_by_sample_cosmx

,Sample Area
Sample ID,
7_2 Cntr,28949809.80
7_3 Cntr,28259154.00
175_2 HD Model,25952368.44
175_3 HD Model,30318066.12


### Vizgen Cell Area Analysis

In [11]:
vizgen_scdata = sc.read_h5ad(r"D:\SMI-0214_DonCleveland_UCSD\SMI-0214_DonCleveland_UCSD\Vizgen Analysis\vizgen_scdata_filtered.h5ad")
vizgen_scdata

AnnData object with n_obs × n_vars = 158616 × 135
    obs: 'total_counts', 'n_genes', 'area', 'volume', 'Sample ID', 'leiden', 'celltype'
    var: 'mean', 'std'
    uns: 'celltype_colors', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'pca', 'rank_genes_groups'
    obsm: 'X_pca', 'X_raw', 'X_spatial', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [21]:
vizgen_df = vizgen_scdata.obs[['Sample ID', 'area']]
vizgen_df = vizgen_df.rename(columns = {'area':'Sample Area'})
vizgen_df

,Sample ID,Sample Area
2,0,875.521977
71,0,119.650625
72,0,1185.556432
73,0,85.271840
74,0,29.316084
...,...,...
199570,0,37.077160
199572,0,54.992407
199573,0,268.149889
199606,0,285.639367


In [22]:
cell_area_by_sample_vizgen = vizgen_df.groupby('Sample ID').sum()
cell_area_by_sample_vizgen

,Sample Area
Sample ID,
0,8.465452e+07
